In [5]:
import torch

from transformer import Transformer, Autobot

In [3]:
trans = Transformer(
    d_model=1,
    nhead=1,
    num_encoder_layers=1,
    num_decoder_layers=1,
    dim_feedforward=4,
    dropout=0.1,
    activation='relu',
    layer_norm_eps=1e-5,
)

In [6]:
zeroones = torch.zeros(100, 1, 1)

In [10]:
trans(zeroones, zeroones).shape

torch.Size([100, 1, 1])